# Importaciones

 Usaremos FastAPI en lugar de FLASK por comodidad ya que FastAPI valida automáticamente que los datos estén en el formato correcto, no bloquea el notebook cuando lanzamos el servidor y en este caso es mas limpio en lo que a código se refiere

In [ ]:
import sys
!{sys.executable} -m pip install fastapi uvicorn nest_asyncio gdown pyngrok --quiet

In [ ]:
import gdown, pickle, base64, io, uuid, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import nest_asyncio
import uvicorn
from threading import Thread
from pyngrok import ngrok
from getpass import getpass
import requests
from PIL import Image
from io import BytesIO

In [ ]:
url_modelo = "https://drive.google.com/uc?export=download&id=1yB477t_Ese8qFf9xQoMg1fXuo9EXQoDH"
output_modelo = "gradboost_model.pkl"

url_pca = "https://drive.google.com/uc?export=download&id=19ah8WNBICdyc0gVK2zUkk5JW-LTUjih5"
output_pca = "pca.pkl"

url_medias = "https://drive.google.com/uc?export=download&id=1Ryh3o3Nq7Tb1LJh4_seSJrxQaMdEilq5"
output_medias = "medias.csv"

# Descargar desde Drive
gdown.download(url_modelo, output_modelo, quiet=False)
gdown.download(url_pca, output_pca, quiet=False)
gdown.download(url_medias, output_medias, quiet=False)

# Cargar el modelo y el scaler
with open(output_modelo, "rb") as f:
    data_cargada = pickle.load(f)
model = data_cargada["model"]
scaler = data_cargada["scaler"]

# Cargar el PCA entrenado
with open(output_pca, "rb") as f:
    pca = pickle.load(f)

# Cargar las coordenadas PCA de las medias generacionales
df_medias = pd.read_csv(output_medias)

print("\nArchivos cargados correctamente")

# Pruebas

In [ ]:
# Ejemplo de usuario nuevo
nuevo_usuario = pd.DataFrame([{
    "celtas": 60,
    "iberos": 30,
    "fenicios": 0,
    "griegos": 0,
    "italicos": 10
}])

# Calcular y mostrar la suma total
suma_total = nuevo_usuario.sum(axis=1).iloc[0]
print(f"Suma total de porcentajes: {suma_total:.2f}%")

In [ ]:
# Escalamos igual que el entrenamiento
usuario_scaled = scaler.transform(nuevo_usuario)

In [ ]:
# Predicción
dist_pred = model.predict(usuario_scaled)[0]
print(f"Distancia con respecto a la Generación 1: {dist_pred:.6f}")

# Proyección PCA
usuario_pca = pca.transform(usuario_scaled)

In [ ]:
plt.figure(figsize=(7, 6))

# Generación ancestral (G1)
plt.scatter(df_medias.iloc[0, 0], df_medias.iloc[0, 1],
            color="red", s=150, marker="X", label="Generación 1 (ancestral)")

# Otras generaciones
plt.scatter(df_medias.iloc[1:, 0], df_medias.iloc[1:, 1],
            color="gray", label="Otras generaciones", alpha=0.7)

# Usuario nuevo
plt.scatter(usuario_pca[0, 0], usuario_pca[0, 1],
            color="blue", s=100, label="Usuario nuevo")

# Etiquetas de generaciones
for j in range(df_medias.shape[0]):
    plt.text(df_medias.iloc[j, 0], df_medias.iloc[j, 1],
             f"G{j+1}", fontsize=9, ha="center", va="center",
             bbox=dict(facecolor="white", alpha=0.6, edgecolor="none", pad=1))

plt.xlabel("Componente principal 1")
plt.ylabel("Componente principal 2")
plt.title("PCA: Generaciones y Usuario Nuevo")
plt.legend()
plt.tight_layout()
plt.show()

# Montaje Servidor

In [ ]:
app = FastAPI(title="API Distancia Genética")

# Definición del input esperado
class DatosUsuario(BaseModel):
    celtas: float
    iberos: float
    fenicios: float
    griegos: float
    italicos: float

@app.post("/predecir/")
def predecir(datos: DatosUsuario):
    # Datos del usuario
    X = np.array([[datos.celtas, datos.iberos, datos.fenicios, datos.griegos, datos.italicos]])

    # Escalar y predecir
    X_scaled = scaler.transform(X)
    distancia = model.predict(X_scaled)[0]

    # Transformar con PCA
    usuario_pca = pca.transform(X_scaled)

    # --- Crear gráfica PCA ---
    plt.figure(figsize=(7,6))

    # Generación ancestral (G1)
    plt.scatter(df_medias.iloc[0,0], df_medias.iloc[0,1],
                color="red", s=150, marker="X", label="Generación 1 (ancestral)")

    # Otras generaciones
    plt.scatter(df_medias.iloc[1:,0], df_medias.iloc[1:,1],
                color="gray", alpha=0.7, label="Otras generaciones")

    # Usuario
    plt.scatter(usuario_pca[0,0], usuario_pca[0,1],
                color="blue", s=100, label="Usuario")

    # Etiquetas de generaciones
    for j in range(df_medias.shape[0]):
        plt.text(df_medias.iloc[j, 0], df_medias.iloc[j, 1],
                f"G{j+1}", fontsize=9, ha="center", va="center",
                bbox=dict(facecolor="white", alpha=0.6, edgecolor="none", pad=1))

    plt.xlabel("Componente principal 1")
    plt.ylabel("Componente principal 2")
    plt.title("PCA: Generaciones y Usuario")
    plt.legend()
    plt.tight_layout()

    # Convertir gráfico a Base64
    buf = io.BytesIO()
    plt.savefig(buf, format="png", dpi=150)
    plt.close()
    buf.seek(0)
    imagen_base64 = base64.b64encode(buf.read()).decode("utf-8")

    # Devolver JSON
    return JSONResponse(content={
        "distancia_g1": round(distancia, 6),
        "pca_x": round(float(usuario_pca[0, 0]), 6),
        "pca_y": round(float(usuario_pca[0, 1]), 6),
        "imagen_base64": imagen_base64
    })

In [ ]:
# Pedir el token
authtoken = getpass("Introduce tu authtoken de ngrok: ")
# Configurar ngrok
!ngrok authtoken {authtoken}

In [ ]:
# Elegir un puerto libre (5000 y 8888 van bien)
PORT = 5000

# Ejecutar el servidor (Uvicorn)
nest_asyncio.apply()

def run():
    uvicorn.run(app, host="0.0.0.0", port=PORT)

# Iniciar FastAPI en un hilo
Thread(target=run).start()

# Abrir túnel público con ngrok
tunnel = ngrok.connect(PORT)
public_url = tunnel.public_url
print("Tu URL pública es:", public_url)

# Solicitud de predicción al Servidor

In [ ]:
url_publica= f"{public_url}/predecir/"

In [ ]:
# Datos de ejemplo para enviar
data = {
    "celtas": 60,
    "iberos": 30,
    "fenicios": 0,
    "griegos": 0,
    "italicos": 10
}

In [ ]:
# Hacer la petición POST al endpoint FastAPI
response = requests.post(url_publica, json=data)
resultado = response.json()

In [ ]:
# Mostrar resultados en Colab
print("Distancia G1:", resultado["distancia_g1"])
print("PCA X:", resultado["pca_x"])
print("PCA Y:", resultado["pca_y"])

In [ ]:
# Convertir la imagen Base64 a imagen y mostrarla
imagen_bytes = base64.b64decode(resultado["imagen_base64"])
imagen = Image.open(BytesIO(imagen_bytes))
imagen = imagen.resize((700, 600))
display(imagen)